In [1]:
#matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import opensfm
import cv2
from opensfm import dataset, features, io
import json

matplotlib.rcParams['figure.figsize'] = (12, 12)

In [2]:
path = '/home/ubuntu/projects/OpenSfM/data/gezer/'
data = dataset.DataSet(path)

In [3]:
camera = data.load_camera_models(); camera

{'v2 dji fc300s 4000 3000 perspective 0.5555': PerspectiveCamera('v2 dji fc300s 4000 3000 perspective 0.5555', 'perspective', 4000, 3000, 0.5555555555555556, 0.0, 0.0, 0.5555555555555556, 0.0, 0.0)}

In [4]:
camera_name = list(camera.keys())[0]

In [5]:
camera = camera[camera_name]; camera

PerspectiveCamera('v2 dji fc300s 4000 3000 perspective 0.5555', 'perspective', 4000, 3000, 0.5555555555555556, 0.0, 0.0, 0.5555555555555556, 0.0, 0.0)

In [6]:
K = camera.get_K_in_pixel_coordinates(); K

array([[2.22222222e+03, 0.00000000e+00, 1.99950000e+03],
       [0.00000000e+00, 2.22222222e+03, 1.49950000e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [ ]:
# notes: normalize_pts as defined in /home/ubuntu/projects/OpenSfM/opensfm/features.py

In [ ]:
class NGRansac:
    
    # init model with default params. Calculate only fundamental matrix
    def __init__(self,
                 frame_size,                 
                 batchsize=32,
#                  fmat=True,
                 hyps=1000,
                 model='',
                 nfeatures=2000,
                 nosideinfo=False,                 
                 orb=False,
                 ratio=1.0,
                 refine=False,
                 resblocks=12,
                 rootsift=False,
                 session='',
                 thershold=0.001):
        
        # load network
        model_file = model
        if len(model_file) == 0:
            model_file = util.create_session_string('e2e', fmat, orb, rootsift, ratio, session)
            model_file = 'models/weights_' + model_file + '.net'            
            print('Loading pre-trained model:', model_file)
            
        model = CNNet(resblocks)
        model.load_state_dict(torch.load(model_file))
        model = model.cuda()
        model.eval()        
        print('model ', model_file, ' successfully loaded.')        
        
        self.model      = model
        self.frame_size = frame_size
        self.nosideinfo = nosideinfo
        self.hyps       = hyps
        self.threshold  = threshold
        self.refine     = refine
        
    
    # pts1 and pts2 must be normalized to the frame size before running the procedure below
    # as well the ratios must be computed
    def findFundamentalMat(self, pts1, pts2, ratios):
        
        # normalize x and y coordinates before passing them to the network
        # normalized by the image size
        util.normalize_pts(pts1, self.frame_size)
        util.normalize_pts(pts2, self.frame_size)
        
        if self.nosideinfo:
            # remove side information before passing it to the network
            ratios = np.zeros(ratios.shape)
        
        # create data tensor of feature coordinates and matching ratios
        correspondences = np.concatenate((pts1, pts2, ratios), axis=2)
        correspondences = np.transpose(correspondences)
        correspondences = torch.from_numpy(correspondences).float()

        # predict neural guidance, i.e. RANSAC sampling probabilities
        log_probs = self.model(correspondences.unsqueeze(0).cuda())[0] #zero-indexing creates and removes a dummy batch dimension
        probs = torch.exp(log_probs).cpu()

        out_model     = torch.zeros((3, 3)).float() # estimated model
        out_inliers   = torch.zeros(log_probs.size()) # inlier mask of estimated model
        out_gradients = torch.zeros(log_probs.size()) # gradient tensor (only used during training)
        rand_seed     = 0 # random seed to by used in C++
        
        # run NG-RANSAC
        # === CASE FUNDAMENTAL MATRIX =========================================

        # undo normalization of x and y image coordinates
        util.denormalize_pts(correspondences[0:2], self.frame_size)
        util.denormalize_pts(correspondences[2:4], self.frame_size)
        
        incount = ngransac.find_fundamental_mat(correspondences, 
                                                probs, 
                                                rand_seed, 
                                                self.hyps, 
                                                self.threshold, 
                                                self.refine, 
                                                out_model, 
                                                out_inliers, 
                                                out_gradients)
        
        print("\n=== Model found by NG-RANSAC: =======\n")
        print("\nNG-RANSAC Inliers: ", int(incount))
        
        # Fundamental matrix
        return out_model.numpy()
